https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb 


In [1]:
import tensorflow as tf
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Embedding
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [2]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [5]:
%cd $PATH_TO_FOLDER

/content/drive/MyDrive/Name2Demographics


In [3]:
%cd ../../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [5]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from char_utils import *

sys.path.insert(0, 'Models/AIEEEGenderData/Preprocessing/')
from aieee_gender import *

In [6]:
aieee_df = AIEEEGenderData()

In [7]:
train_x, train_y, val_x, val_y, test_x, test_y = splitData(aieee_df, 0.7, 0, 'Name', 'char_cnn_cbse_oov.txt')

(train:(287000, 4), val:(0, 0), test:(122286, 4))
Train: ((281357, 30),(281357, 2)) Val:((0,),(0,)) Test: ((119569, 30),(119569, 2))


In [8]:
voc = vocab()

#### https://stackoverflow.com/questions/59265920/this-tensorflow-binary-is-optimized-with-intelr-mkl-dnn-to-use-the-following-c


In [9]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [10]:
inputs = Input(MAXLEN_NAME_CHAR_CNN)
x = Embedding(len(voc), len(voc))(inputs)
x = Conv1D(filters = 256, kernel_size = 7, activation = 'relu')(x)
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

2022-03-17 14:16:25.208677: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-17 14:16:25.209242: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: obelix
2022-03-17 14:16:25.209278: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: obelix
2022-03-17 14:16:25.209558: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-03-17 14:16:25.209962: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-03-17 14:16:25.209990: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-03-17 14:16:25.573888: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 embedding (Embedding)       (None, 30, 28)            784       
                                                                 
 conv1d (Conv1D)             (None, 24, 256)           50432     
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 2)                 12290     
                                                                 
Total params: 63,506
Trainable params: 63,506
Non-trainable params: 0
_________________________________________________________________


In [12]:
batch_size=1000
model.fit(train_x, train_y,batch_size=batch_size,epochs=15,validation_data=(test_x, test_y))

2022-03-17 14:35:26.001081: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 67525680 exceeds 10% of free system memory.


Epoch 1/15
282/282 [==============================] - 29s 83ms/step - loss: 0.4055 - accuracy: 0.8102 - val_loss: 0.3315 - val_accuracy: 0.8597
Epoch 2/15
282/282 [==============================] - 16s 56ms/step - loss: 0.2722 - accuracy: 0.8894 - val_loss: 0.2890 - val_accuracy: 0.8812
Epoch 3/15
282/282 [==============================] - 15s 55ms/step - loss: 0.2279 - accuracy: 0.9113 - val_loss: 0.2594 - val_accuracy: 0.8952
Epoch 4/15
282/282 [==============================] - 18s 62ms/step - loss: 0.2015 - accuracy: 0.9228 - val_loss: 0.2391 - val_accuracy: 0.9052
Epoch 5/15
282/282 [==============================] - 15s 55ms/step - loss: 0.1828 - accuracy: 0.9310 - val_loss: 0.2285 - val_accuracy: 0.9101
Epoch 6/15
282/282 [==============================] - 16s 56ms/step - loss: 0.1695 - accuracy: 0.9363 - val_loss: 0.2240 - val_accuracy: 0.9125
Epoch 7/15
282/282 [==============================] - 16s 55ms/step - loss: 0.1591 - accuracy: 0.9405 - val_loss: 0.2174 - val_accuracy:

In [13]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

3737/3737 [==============================] - 8s 2ms/step - loss: 0.2018 - accuracy: 0.9238
Test score: 0.20184087753295898
Test accuracy: 0.9237595200538635


In [ ]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

In [19]:
filename = "Models/AIEEE/CharCNN/SavedModel/CharCNN"
model.save(filename)

INFO:tensorflow:Assets written to: Models/CBSEData/CharCNN/SavedModel/CharCNN/assets
